In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling

# CSV 파일 경로
labeled_data_csv = "./origin_data/train.csv" # 레이블이 있는 데이터 파일
unlabeled_data_csv = "./data/unlabeld_data.csv"  # 레이블이 없는 데이터 파일
test_csv = "./origin_data/test.csv"

# CSV 파일 읽기
labeled_data = pd.read_csv(labeled_data_csv).drop(columns=["LogD", "Molecular_PolarSurfaceArea"])
unlabeled_data = pd.read_csv(unlabeled_data_csv)
test_data = pd.read_csv(test_csv).drop(columns=["LogD", "Molecular_PolarSurfaceArea"])

# 레이블이 있는 데이터 추출
X_labeled = labeled_data.drop(columns=['MLM', 'HLM'])
Y_labeled = labeled_data[['MLM', 'HLM']]

# 레이블이 없는 데이터 추출
X_unlabeled = unlabeled_data

# 초기 학습 데이터 선택 (랜덤으로 선택)
initial_idx = np.random.choice(range(len(X_labeled)), size=10, replace=False)
X_initial = X_labeled.iloc[initial_idx]
Y_initial = Y_labeled.iloc[initial_idx]

# 초기 모델 학습
learner = ActiveLearner(
    estimator=RandomForestRegressor(),
    query_strategy=uncertainty_sampling,
    X_training=X_initial,
    y_training=Y_initial
)

# Active Learning 반복 과정
n_queries = 20
for i in range(n_queries):
    query_idx, query_instance = learner.query(X_unlabeled)
    learner.teach(X_unlabeled.iloc[query_idx], Y_unlabeled.iloc[query_idx])
    
    # 쿼리된 데이터는 학습 데이터에서 제거
    X_unlabeled = X_unlabeled.drop(X_unlabeled.index[query_idx])
    Y_unlabeled = Y_unlabeled.drop(Y_unlabeled.index[query_idx])

Y_test_pred = learner.predict(test_data)



ModuleNotFoundError: No module named 'modAL'